In [2]:
pip install numpy pandas matplotlib seaborn datasets tensorflow scikit-learn

     --------------------------------------- 11.6/11.6 MB 15.2 MB/s eta 0:00:00
     ---------------------------------------- 7.8/7.8 MB 11.4 MB/s eta 0:00:00
     ------------------------------------- 294.9/294.9 KB 17.8 MB/s eta 0:00:00
     ------------------------------------- 491.2/491.2 KB 30.1 MB/s eta 0:00:00
     --------------------------------------- 11.2/11.2 MB 27.3 MB/s eta 0:00:00
     ------------------------------------- 509.2/509.2 KB 16.1 MB/s eta 0:00:00
     ------------------------------------- 347.8/347.8 KB 21.1 MB/s eta 0:00:00
     ---------------------------------------- 2.7/2.7 MB 28.7 MB/s eta 0:00:00
     ---------------------------------------- 111.1/111.1 KB ? eta 0:00:00
     ---------------------------------------- 2.2/2.2 MB 28.1 MB/s eta 0:00:00
     ------------------------------------- 211.8/211.8 KB 12.6 MB/s eta 0:00:00
     ---------------------------------------- 55.8/55.8 KB 3.0 MB/s eta 0:00:00
     ------------------------------------- 481.4

You should consider upgrading via the 'C:\Users\tamer\OneDrive - Høgskulen på Vestlandet\Dokumenter\3 År\DAT255 Deep learning\toxicity2\toxicityPython39\Scripts\python.exe -m pip install --upgrade pip' command.


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
from datasets import load_dataset
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix

In [4]:
# Load civil_comments dataset
dataset = load_dataset("google/civil_comments")
print(dataset['train'][0])

{'text': "This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!", 'toxicity': 0.0, 'severe_toxicity': 0.0, 'obscene': 0.0, 'threat': 0.0, 'insult': 0.0, 'identity_attack': 0.0, 'sexual_explicit': 0.0}


In [5]:
df = pd.DataFrame(dataset['train'])
df.head()

,text,toxicity,severe_toxicity,obscene,threat,insult,identity_attack,sexual_explicit
0,"This is so cool. It's like, 'would you want yo...",0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0
1,Thank you!! This would make my life a lot less...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0
2,This is such an urgent design problem; kudos t...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0
3,Is this something I'll be able to install on m...,0.000000,0.000000,0.0,0.0,0.00000,0.000000,0.0
4,haha you guys are a bunch of losers.,0.893617,0.021277,0.0,0.0,0.87234,0.021277,0.0


In [6]:
from tensorflow.keras.layers import TextVectorization

comment_text = df['text']
lables = df[df.columns[2:]].values

print(comment_text)
print(lables)

0          This is so cool. It's like, 'would you want yo...
1          Thank you!! This would make my life a lot less...
2          This is such an urgent design problem; kudos t...
3          Is this something I'll be able to install on m...
4                       haha you guys are a bunch of losers.
                                 ...                        
1804869    Maybe the tax on "things" would be collected w...
1804870    What do you call people who STILL think the di...
1804871    thank you ,,,right or wrong,,, i am following ...
1804872    Anyone who is quoted as having the following e...
1804873    Students defined as EBD are legally just as di...
Name: text, Length: 1804874, dtype: object
[[0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.         0.        ]
 ...
 [0.         0.         0.         0.         0.         0.        ]
 [0.0

In [7]:
vectorizer = TextVectorization(max_tokens=10000, output_sequence_length=1800, output_mode='int')
vectorizer.adapt(comment_text.values)
vectorized_text = vectorizer(comment_text.values)

ResourceExhaustedError: Exception encountered when calling TextVectorization.call().

[1m{{function_node __wrapped__RaggedTensorToTensor_num_row_partition_tensors_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[1804874,1800] and type int64 on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:RaggedTensorToTensor] name: [0m

Arguments received by TextVectorization.call():
  • inputs=array(["This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!",
       "Thank you!! This would make my life a lot less anxiety-inducing. Keep it up, and don't let anyone get in your way!",
       'This is such an urgent design problem; kudos to you for taking it on. Very impressive!',
       ..., 'thank you ,,,right or wrong,,, i am following your advice',
       'Anyone who is quoted as having the following exchange, even if apocryphal, would have received my vote! \n\nBessie Braddock: "Winston

In [ ]:
print(len(df))

In [ ]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, lables))
dataset = dataset.cache()
dataset = dataset.shuffle(1804874)
dataset = dataset.batch(16)
dataset = dataset.prefetch(8) # helps bottlenecks